In [1]:
import pandas as pd

COLUMN_NAMES = ['Datapoint', 'Concepts', 'Accuracy', 'Overlap', 'Time']

# df = pd.DataFrame(data=[], columns=COLUMN_NAMES)
# temp_df = pd.read_csv("wl_all_2_results_raw.csv", names=COLUMN_NAMES)

lang_df = pd.read_csv("commons-lang-synth.csv", names=COLUMN_NAMES)
math_df = pd.read_csv("commons-math-synth.csv", names=COLUMN_NAMES)
joda_df = pd.read_csv("joda-time-synth.csv", names=COLUMN_NAMES)

lang_df['Project'] = "Lang"
math_df['Project'] = "Math"
joda_df['Project'] = "Joda"

df = pd.concat([lang_df, math_df, joda_df], ignore_index=True, sort=False)
df['Method'] = "Heddle"

project = "Lang"
method = "Heddle"

# temp_df['Project'] = pd.Series([project]*len(temp_df.index))
# temp_df['Method'] = pd.Series([method]*len(temp_df.index))

# df = df.append(temp_df, sort=False)

df = df[df.Accuracy.notna()]
# Sadly we have too few examples of succesful 4-commit-tangles for results
# to be statistically meaningful (len(df[df.Concepts == 4]) == 10)
df = df[df.Concepts < 4]

# This avoid a div by 0 later, but this also means we have to manually set the ylim in plots
df["Time"] = df.apply(lambda row: row["Time"] if row["Time"] > 0 else row["Time"] + 1e-31,
                      axis=1)
df

,Datapoint,Concepts,Accuracy,Overlap,Time,Project,Method
0,b2f1757bf9ec1632a940b9a2e65a1a022ba54af8_b2bba...,2,0.486838,0.463512,11992.026108,Lang,Heddle
1,7f89bd4850b7a5b0dd2a10d2f2daa62805de5bca_7e512...,1,1.000000,NaN,0.023416,Lang,Heddle
2,7f89bd4850b7a5b0dd2a10d2f2daa62805de5bca_e4b58...,2,0.400208,0.360222,1121.999110,Lang,Heddle
3,7f89bd4850b7a5b0dd2a10d2f2daa62805de5bca_0ac77...,3,0.465347,0.433896,1260.897975,Lang,Heddle
4,e4b58afe49b8232e0c7a1e051a6cb8580b14d6b6_0ac77...,2,0.458333,0.277778,2.921547,Lang,Heddle
...,...,...,...,...,...,...,...
62,7db787317115fc706deb42cdc623d0b5324375d0_ec6e7...,2,0.460733,0.406806,981.921540,Joda,Heddle
63,7db787317115fc706deb42cdc623d0b5324375d0_a7a93...,3,0.446994,0.404455,1718.845119,Joda,Heddle
64,9be879e558515f40ab79ce55944919f370f5c865_c52cb...,2,0.703704,NaN,0.673299,Joda,Heddle
65,ea67740bc7bdfe700ec02bb27e5670541b1ecbf2_75190...,2,0.473279,0.450378,5168.310653,Joda,Heddle


In [3]:
df.groupby(['Project', 'Concepts']).median()

/var/folders/95/72xs2mfn5tv8sgfyd51fcnnr0000gn/T/ipykernel_17773/551725955.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby(['Project', 'Concepts']).median()


Accuracy   Overlap         Time
Project Concepts                                 
Joda    1         0.853535  0.535354   118.966689
        2         0.480829  0.396253   861.797508
        3         0.446994  0.380096  1718.845119
Lang    1         0.605697  0.476744     8.780415
        2         0.449167  0.314794    41.735858
        3         0.442308  0.328125   149.150950
Math    1         0.410053  0.325775  3628.952273
        2         0.475494  0.400564   189.655388

In [4]:
results_df = df[df.Concepts > 1].drop(["Overlap"],axis=1).copy()


temp_df = results_df[results_df.Method == method].drop(["Project"], axis = 1)
temp_df.index = range(len(temp_df.index))
temp_df['Project'] = pd.Series(["Overall"]*len(temp_df.index), index=temp_df.index)
results_df = results_df.append(temp_df, ignore_index=True, sort=False)

temp_df = results_df[results_df.Method == method].drop(["Concepts"], axis = 1)
temp_df.index = range(len(temp_df.index))
temp_df['Concepts'] = pd.Series([-1]*len(temp_df.index))
results_df = results_df.append(temp_df, ignore_index=True, sort=False)

results_df.groupby(["Method", "Project", "Concepts"]).median()

/var/folders/95/72xs2mfn5tv8sgfyd51fcnnr0000gn/T/ipykernel_17773/3095149599.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(temp_df, ignore_index=True, sort=False)
/var/folders/95/72xs2mfn5tv8sgfyd51fcnnr0000gn/T/ipykernel_17773/3095149599.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(temp_df, ignore_index=True, sort=False)
/var/folders/95/72xs2mfn5tv8sgfyd51fcnnr0000gn/T/ipykernel_17773/3095149599.py:14: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  results_df.groupby(["Method", "Project", "Concepts"]).median()


Accuracy         Time
Method Project Concepts                       
Heddle Joda    -1        0.463799   864.367437
                2        0.480829   861.797508
                3        0.446994  1718.845119
       Lang    -1        0.442308    48.374516
                2        0.449167    41.735858
                3        0.442308   149.150950
       Math    -1        0.475494   189.655388
                2        0.475494   189.655388
       Overall -1        0.458333   149.150950
                2        0.465318   125.105787
                3        0.444651   252.746992